<a href="https://colab.research.google.com/github/KoreTane/Analise-GLP/blob/main/AnalyserPowerBI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.3 MB/s eta 0:00:00


In [7]:
import pandas as pd
import json

### Carregar o arquivo JSON
with open('PowerBIPerformanceDataANTES.json', 'r', encoding='utf-8-sig') as file:
    dados_json = json.load(file)

# Converter os dados em um DataFrame
df = pd.DataFrame(dados_json)

# Normalizar a coluna 'events'
events_df = pd.json_normalize(df['events'])
df_final = pd.concat([df.drop(columns=['events']), events_df], axis=1)

# Imprimir os nomes das colunas para verificar
print("Nomes das colunas:", df_final.columns.tolist())

# Selecionar as colunas desejadas (ajustando para os nomes corretos)
colunas_desejadas = ['version', 'name', 'component', 'metrics.sourceLabel',
                     'metrics.visualType', 'start', 'end']

# Verificar se todas as colunas desejadas estão presentes
for coluna in colunas_desejadas:
    if coluna not in df_final.columns:
        print(f"A coluna '{coluna}' não está presente no DataFrame.")

# Tentar selecionar as colunas desejadas
df_selected = df_final.loc[:, colunas_desejadas]

# Converter colunas de data (removendo timezone)
df_selected['start'] = pd.to_datetime(df_selected['start'], errors='coerce').dt.tz_localize(None)
df_selected['end'] = pd.to_datetime(df_selected['end'], errors='coerce').dt.tz_localize(None)

# Preencher valores nulos (opcional)
df_selected.fillna('Não Especificado', inplace=True)

# Renomear colunas para melhor legibilidade
df_selected.columns = ['Versão', 'Nome', 'Componente', 'Fonte de Métricas',
                       'Tipo Visual', 'Início', 'Fim']

# Exportar para um arquivo Excel
output_file = 'PowerBIPerformanceData.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    df_selected.to_excel(writer, index=False, sheet_name='Dados')

    # Acessar o workbook e a worksheet
    workbook  = writer.book
    worksheet = writer.sheets['Dados']

    # Definir formato de cabeçalho
    header_format = workbook.add_format({
        'bold': True,
        'font_color': 'white',
        'bg_color': '#4F81BD',
        'border': 1,
        'align': 'center',
        'valign': 'vcenter'
    })

    # Aplicar o formato ao cabeçalho
    for col_num, value in enumerate(df_selected.columns.values):
        worksheet.write(0, col_num, value, header_format)

    # Ajustar largura das colunas
    for i in range(len(df_selected.columns)):
        worksheet.set_column(i, i, 20)  # Ajusta a largura para 20 unidades

print(f"Arquivo exportado com sucesso: {output_file}")

Nomes das colunas: ['version', 'sessionId', 'name', 'component', 'start', 'id', 'metrics.sourceLabel', 'end', 'metrics.status', 'metrics.visualTitle', 'metrics.visualId', 'metrics.visualType', 'parentId', 'metrics.QueryText', 'metrics.RowCount', 'metrics.initialLoad']


<ipython-input-7-f7fdf7463101>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Não Especificado' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df_selected.fillna('Não Especificado', inplace=True)


Arquivo exportado com sucesso: PowerBIPerformanceData.xlsx
